# <center> Discrete Fourier Transform and Fast Fourier Transform

In [2]:
import numpy as np

### Create and Verify our Discrete Fourier Transform Function

The Discrete Fourier Transform is a mathematical operation that translates a signal from its configuration space to frequency space. It is useful for decomposing a signal consisting of multiple pure frequencies and for denoising that signal to determine what those pure frequencies are.

$$
X_k =\sum \limits _{n=0} ^{N-1} x_n * e^{-2i \pi k n /N}
$$

- N = number of samples
- n = current sample
- k = current frequency
- $ x_n $ = the sine value at the current sample n
- $ X_k $ = the discrete fourier transform

In [44]:
def DFT(x):
    x = np.asarray(x, dtype=float) # convert to numpy array
    N = len(x) # get the size of our array
    n = np.arange(N) # n goes from 0 to N-1
    k = n.reshape((N, 1)) # shape into array
    e = np.exp(-2j * np.pi * k * n / N) # e^(-2i * pi * k * n/ N)
    X = np.dot(e, x) # dot product
    return X

In [45]:
x = np.random.random(1024) # create a random 1D array
np.fft.fft(x) # apply numpy's fourier transform function

array([512.57377769 +0.j        ,  -1.89537343 +8.67011598j,
        -1.75519176 -5.80395222j, ...,  -4.10693644+15.55178092j,
        -1.75519176 +5.80395222j,  -1.89537343 -8.67011598j])

In [46]:
DFT(x) # apply our fourier transform function

array([512.57377769 +0.j        ,  -1.89537343 +8.67011598j,
        -1.75519176 -5.80395222j, ...,  -4.10693644+15.55178092j,
        -1.75519176 +5.80395222j,  -1.89537343 -8.67011598j])

In [47]:
np.allclose(DFT(x), np.fft.fft(x)) # check that the outputs are in fact the same

True

In [48]:
# compare the time it takes for each to run
%timeit DFT(x)
%timeit np.fft.fft(x)

48.2 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
20.9 µs ± 1.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Create and Verify our Fast Fourier Transform
DFT can be quite slow so we can use the FFT, which exploits the symmetries of our problem, to compute the transform much more quickly.

The Discrete Fourier Transform has a symmetry property, such that $ X_N+k = X_k $ which then extends to $ X_N*i+k = X_k $.  
<em> Note: $ e^{-2i \pi (N+k) n /N} = 1 $
$$
X_N+k = \sum \limits _{n=0} ^{N-1} x_n * e^{-2i \pi (N+k) n /N}
$$
$$
= \sum \limits _{n=0} ^{N-1} x_n * e^{-2i \pi N n /N} * e^{-2i \pi k n /N}
= \sum \limits _{n=0} ^{N-1} x_n * e^{-2i \pi n} * e^{-2i \pi k n /N}
$$
$$
= \sum \limits _{n=0} ^{N-1} x_n * e^{-2i \pi k n /N} = X_k
$$

According to the Cooley-Tukey algorithm, we can split the DFT into two smaller parts, namely, the odds and the evens.
$$
X_k =\sum \limits _{n=0} ^{N-1} x_n * e^{-2i \pi k n /N}
$$
$$
=\sum \limits _{m=0} ^{N/2 -1} x_{2m} * e^{-2i \pi k (2m) /N} + \sum \limits _{m=0} ^{N/2 -1} x_{2m+1} * e^{-2i \pi k (2m+1) /N}
$$
$$
=\sum \limits _{m=0} ^{N/2 -1} x_{2m} * e^{-2i \pi k m /(N/2)} + e^{-2i \pi k /N} + \sum \limits _{m=0} ^{N/2 -1} x_{2m+1} * e^{-2i \pi k (2m+1) /N}
$$

The smaller terms have half the size of our original dataset (N/2) and we can continue to chop them into two, creating smaller and smaller groups. Then, due to our symmetry property, within these two split parts we know that we only need to calculate half of the fields in each term. 
So long as N is a power of 2, the maximum number of times you can split into two equal halves is given by p = log(N).
This speeds up the fourier transform from $ O(N^{2}) $ to $ O(N*\log(N)) $. The N comes from the actual Fourier transform calculations and the log(N) is the number of times we split our dataset, thus how many times we need to perform N calculations.
The FFT algorithm is significantly faster than the direct implementation. However, it still lags behind the numpy implementation by quite a bit. One reason for this is the fact that the numpy implementation uses matrix operations to calculate the Fourier Transforms simultaneously.

In [11]:
def FFT(x):
    """ A recursive implementation of the 1D Cooley-Tukey FFT.
    Requires input with a length that is a power of 2.  """
    x = np.asarray(x, dtype=float) #make into an array
    N = len(x) #get length of our data
    
    if N == 1: # no longer a power of 2
        return x
    else:
        X_even = FFT(x[::2]) #recursively call FFT on every other element starting at 0 (so the evens)
        X_odd = FFT(x[1::2]) #recursively call FFT on every other element starting at 1 (so the odds)
        factor = np.exp(-2j * np.pi * np.arange(N) / N) #create an array of our multiplying factor
        #combine two halves back into a full array
        #the DFT used the dot product between full x array and the multiplier (aka sum of product of corresponding components)
        #here we do the equivalent and get the correct multiplier from our array for it
        X = np.concatenate([X_even + factor[:int(N/2)] * X_odd, X_even + factor[int(N/2):] * X_odd])
        return X

In [50]:
x = np.random.random(1024)
np.allclose(FFT(x), np.fft.fft(x))

True

In [51]:
%timeit DFT(x)
%timeit FFT(x)
%timeit np.fft.fft(x)

53.1 ms ± 6.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.5 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
21 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### FFT speedups
There are other FFT algorithms, such as the prime-factor FFT algorithm and there is also the opportunity to use parallel processing for further speedup.